# Import Statements

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Data Preprocessing

In [ ]:
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d mlg-ulb/creditcardfraud

In [ ]:
# ! unzip /content/creditcardfraud.zip

## Preprocessing

In [ ]:
df = pd.read_csv('./creditcard.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# checking null values
df.info()

In [ ]:
# checking dublicate values
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['Class'].value_counts()

In [ ]:
fraud = df[df['Class'] == 0]
ham = df[df['Class'] == 1]

In [ ]:
dataframe = fraud.sample(473, ignore_index=True)

In [ ]:
dataframe = pd.concat([dataframe, ham], ignore_index=True)

In [ ]:
dataframe.shape

In [ ]:
# splitting the data into indepandent and depandent variable
x = dataframe.drop(columns=['Time', 'Class'], axis=1)
y = dataframe['Class'].values

In [ ]:
# Standardlizing the dataframe
sc = StandardScaler().fit(x)
x = sc.transform(x)

In [ ]:
# splitting the data into testing and training datasets

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [ ]:
x_train.shape, x_test.shape

In [ ]:
# reshaping the independent datasets
x_train = x_train.reshape(709, 29, 1)
x_test = x_test.reshape(237, 29, 1)

# Building and Evaluating the CNN model

In [ ]:
model = keras.models.Sequential()

In [ ]:
# adding 1st cnn layers
model.add(keras.layers.Conv1D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=(29, 1)))

# adding batch normalization layers
model.add(keras.layers.BatchNormalization())

# adding maxpool layers
model.add(keras.layers.MaxPool1D(pool_size=2))

# adding dropout
model.add(keras.layers.Dropout(0.2))

In [ ]:
# adding 2nd cnn layers
model.add(keras.layers.Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'))

# adding batch normalization layers
model.add(keras.layers.BatchNormalization())

# adding maxpool layers
model.add(keras.layers.MaxPool1D(pool_size=2))

# adding dropout
model.add(keras.layers.Dropout(0.4))

In [ ]:
# adding Flatting the layers
model.add(keras.layers.Flatten())

In [ ]:
# adding fully connected layers
model.add(keras.layers.Dense(units=64, activation='relu'))


# output layers
model.add(keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the data

In [ ]:
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

In [ ]:
# Evaluation
loss, score = model.evaluate(x_train, y_train)

In [ ]:
print("The training score: ", score)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
pred = np.where(y_pred > 0.5, 1,0)

In [ ]:
print('Testing  Score:', accuracy_score(y_test, pred))

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
print(confusion_matrix(y_test, pred))